In [7]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY=os.getenv("LAB_OPEN_ROUTER_API_KEY_OLD")

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=API_KEY,
)

def patient_risk_simulation(model, prompt):
    completion = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": (
                    "Assign a numerical risk score from 0–10 (10 being the highest risk) " 
                    "for hospitalization for a person with the following characteristics: "
                    f"{prompt}. "
                    "Respond only with:\n"
                    "1. Risk Assessment Score\n"
                    "2. A list of every single parameter and their associated numerical positive or negative risk values "
                    "that add up to the Risk Assessment Score\n"
                    "3. A one to two sentence rationale.\n"
                    "Exclude baseline risk in all calculations."
                )
            }
        ]
    )
    
    return completion.choices[0].message.content

In [8]:
import os
import pandas as pd

df = pd.read_csv("synthetic_data1.csv")

#change these
num_patients = 50
sim_reps = 2
folder_name = 'full_sim_run_1'

try:
    os.mkdir(folder_name)
    print(f"Folder '{folder_name}' created.")
except FileExistsError:
    print(f"Folder '{folder_name}' already exists.")


results = []
for i in range(num_patients):
  for j in range(sim_reps):
    out = {}
    out["Patient_ID"] = i
    out["Simulation Number"] = j
    row = df.iloc[i]
    prompt = ", ".join([f"{col}: {row[col]}" for col in df.columns])
    result = patient_risk_simulation("openai/gpt-oss-20b:free", prompt)

    #write each file to a text file for reference
    file_path = os.path.join(folder_name, f"output_{i}.txt")
    with open(file_path, "w") as f:
        f.write(result)

    #parse out the result
    out = {} #Why is this here--dont understand purpose 
    text = result.replace("–", "-").strip()
    lines = [line.strip() for line in text.splitlines()]

    # --- 1. Find Risk Assessment Score ---
    score_val = None
    for i, line in enumerate(lines):
        if "Risk Assessment Score" in line:
            # Look at following few lines for a number
            for j in range(i+1, min(i+4, len(lines))):
                candidate = lines[j].replace("*", "").strip()
                try:
                    score_val = float(candidate)
                    out["Risk_Assessment_Score"] = score_val
                    break
                except ValueError:
                    continue
            break
    if "Risk_Assessment_Score" not in out:
        out["Risk_Assessment_Score"] = None  # fallback

    # --- 2. Parse parameter table ---
    # Identify where the table begins/ends
    start = None
    for i, line in enumerate(lines):
        if "Parameter" in line and "Value" in line:
            start = i + 1
            break

    if start:
        for line in lines[start:]:
            if not line.startswith("|"):
                # table ended
                break

            parts = [p.strip() for p in line.split("|") if p.strip()]

            if len(parts) == 2:
                name, val = parts
                try:
                    out[name] = float(val)
                except ValueError:
                    pass  # ignore non-numeric rows

    # --- 3. Extract rationale ---
    rationale_text = ""
    found_rationale = False
    for i, line in enumerate(lines):
        if "Rationale" in line:
            found_rationale = True
            continue
        if found_rationale:
            rationale_text += line + " "
    out["Rationale"] = rationale_text.strip()

    results.append(out)
    print(out)

pd.DataFrame(out, index=range(i*j)).to_csv("parsed_output_fill_in_nones.csv")

Folder 'full_sim_run_1' already exists.


NotFoundError: Error code: 404 - {'error': {'message': 'No endpoints found matching your data policy (Free model publication). Configure: https://openrouter.ai/settings/privacy', 'code': 404}}